In [3]:
import pathlib
import pandas as pd
import pickle as pkl
import collections
import gzip

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"

_PROJECT_CONFIG_NB = str(pathlib.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

_MYNAME="cache-busco-stats"
_MYSTAMP=get_nb_stamp(_MYNAME)

# created with
# NB::utils::infer-sex-specific-busco
sex_specific_genes_file = PROJECT_BASE.joinpath(
    "annotations", "autogen", "odb10_primates.sex-specific-genes.female.txt"
)
sex_specific_genes = set(open(sex_specific_genes_file).read().strip().split())

karyotype_summary = {
    "verkko": PROJECT_DATA_ROOT.joinpath("2023_karyo_est/hgsvc3/verkko/karyo-est.hgsvc3-verkko.tsv"),
    "hifiasm": PROJECT_DATA_ROOT.joinpath("2023_karyo_est/hgsvc3/hifiasm/karyo-est.hgsvc3-hifiasm.tsv")
}

data_folder_root = PROJECT_DATA_ROOT.joinpath(
    "2024_busco/per_sample"
)

def read_karyo_file(file_path):
    df = pd.read_csv(file_path, sep="\t", header=0, skiprows=1)
    karyo_lut = dict(
        ((row.sample, row.asm_unit.split("-")[1]), row.karyotype) for row in df.itertuples()
    )
    return karyo_lut


def correct_phasing_issues(df):
    pass
        

def correct_sex_bias(table):
    pass   


def read_busco_merged_table(file_path, au_sex, sex_genes):

    df = pd.read_csv(file_path, sep="\t", header=0)
    sample = file_path.name.rsplit(".", 5)[0]
    total_genes = df.shape[0]

    renamer = dict()
    for c in df.columns[1:]:
        if "hap1" in c:
            renamer[c] = "hap1"
        if "hap2" in c:
            renamer[c] = "hap2"
        if "unassigned" in c:
            renamer[c] = "hap0"
    df.rename(renamer, axis=1, inplace=True)
    
    summ_hap1 = df["hap1"].value_counts()
    summ_hap2 = df["hap2"].value_counts()
    summ_un = None
    if df.shape[1] == 4:
        summ_un = df["hap0"].value_counts()
        summ_un["is_male"] = -1
        summ_un["adj_total"] = total_genes

    h1_sex = au_sex[(sample, "hap1")]
    h2_sex = au_sex[(sample, "hap2")]

    if h1_sex != h2_sex and h1_sex != "any" and h2_sex != "any":
        adj_card = total_genes - sex_genes
        applies_to = "hap1" if h1_sex == "male" else "hap2"
    else:
        adj_card = total_genes
        applies_to = "no-bias"

    if applies_to == "hap1":
        summ_hap1["adj_total"] = adj_card
        summ_hap1["is_male"] = 1
        summ_hap1["Missing"] -= sex_genes
        
        summ_hap2["adj_total"] = total_genes
        summ_hap2["is_male"] = 0
    elif applies_to == "hap2":
        summ_hap2["adj_total"] = adj_card
        summ_hap2["is_male"] = 1
        summ_hap2["Missing"] -= sex_genes
        
        summ_hap1["adj_total"] = total_genes
        summ_hap1["is_male"] = 0
    else:
        summ_hap1["adj_total"] = adj_card
        summ_hap1["is_male"] = 0
        summ_hap2["adj_total"] = adj_card
        summ_hap2["is_male"] = 0
        

    merged_stats = pd.concat(
        [summ_hap1, summ_hap2],
        axis=1, ignore_index=False,
    )
    merged_stats.columns = ["hap1", "hap2"]
    if summ_un is not None:
        merged_stats = pd.concat(
            [merged_stats, summ_un],
            axis=1, ignore_index=False
        )
        merged_stats.columns = ["hap1", "hap2", "unassigned"]

    merged_stats = merged_stats.transpose()
    merged_stats.columns = [c.lower() for c in merged_stats.columns]

    for column in ["single", "missing", "duplicated", "fragmented"]:
        merged_stats[column] = merged_stats[column].fillna(0, inplace=False)
        merged_stats[column + "_pct"] = (merged_stats[column] / merged_stats["adj_total"] * 100).round(2)
    merged_stats["total"] = total_genes


    multi_index = []
    for row in merged_stats.itertuples():
        if row.is_male > 0:
            karyo = "male"
        elif row.is_male < 0:
            karyo = "unk"
        else:
            karyo = "female"
        multi_index.append((sample, row.Index, karyo))
    mi = pd.MultiIndex.from_tuples(multi_index, names=["sample", "asm_unit", "karyotype"])
    merged_stats = merged_stats.set_index(mi, inplace=False)
    merged_stats.drop("is_male", axis=1, inplace=True)
    return merged_stats
        

for assembler in ["verkko", "hifiasm"]:
    karyo_lut = read_karyo_file(karyotype_summary[assembler])
    data_folder = data_folder_root.joinpath(assembler)
    full_dataset = []
    for table_file in data_folder.glob("*.tsv.gz"):
        print(table_file.name)
        stats = read_busco_merged_table(table_file, karyo_lut, len(sex_specific_genes))
        full_dataset.append(stats)
    full_dataset = pd.concat(full_dataset, axis=0, ignore_index=False)
    full_dataset["delta_n"] = 0
    full_dataset.loc[full_dataset["adj_total"] < 13780, "delta_n"] = len(sex_specific_genes)
    
    for column in ["single", "missing", "duplicated", "fragmented", "adj_total"]:
        full_dataset[column] = full_dataset[column].astype(int)
    
    full_dataset.sort_index()

    cache_table = PROJECT_NB_CACHE.joinpath(
        f"compleasm-{assembler}.cache.tsv.gz"
    )
    with gzip.open(cache_table, "wt") as dump:
        _ = dump.write(f"# {_MYSTAMP}\n")
        full_dataset.to_csv(dump, sep="\t", header=True, index=True)
        



HG00358.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG02059.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA19238.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA24385.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG00514.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA18989.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG00171.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA19036.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG00733.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG01457.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG03732.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG01114.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG02666.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG04217.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA19331.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG01573.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG03520.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
HG02554.vrk-ps-sseq.busco.primates_odb10.merged.tsv.gz
NA20847.vr